In [1]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
from matplotlib.ticker import FuncFormatter 
#This will be used to create the pre planned events icon
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox) 
from matplotlib._png import read_png 
from matplotlib.patches import Ellipse
import matplotlib.mlab as mlab
%matplotlib inline
import plotly.plotly as py
import seaborn as sns
import xml.etree.ElementTree as ET
import geopy.distance
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import datetime
import time 
import folium
from folium import plugins
sns.set()
pal = sns.hls_palette(10, h=.5)
sns.set_palette(pal)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from matplotlib.ticker import FuncFormatter 
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox) 
from matplotlib._png import read_png
from matplotlib.patches import Ellipse
import matplotlib.mlab as mlab
import plotly.plotly as py

import networkx as nx
try:
    import pygraphviz
    from networkx.drawing.nx_agraph import graphviz_layout
except ImportError:
    try:
        import pydot
        from networkx.drawing.nx_pydot import graphviz_layout
    except ImportError:
        raise ImportError("import failed")

In [2]:
path ='Datasets/Trips_Data/2017TripsData'

trips_2017 = glob.glob(path + "/*.csv")
trips_data_frame = pd.DataFrame()
columns_name = ['rental_id','duration','bike_id','end_date','end_station_id','end_station_name','start_date',
             'start_station_id','start_station_name']
data_frame_lists = []

columns=list(range(9))
for read_trip in trips_2017:
    
    data_frame = pd.read_csv(read_trip,names=columns_name,usecols=columns,header=0, engine='python',
                     parse_dates=['start_date'],dayfirst=True,infer_datetime_format=True)
    data_frame_lists.append(data_frame)
trips_data_frame=pd.concat(data_frame_lists,ignore_index=True)
trips_data_frame = trips_data_frame.dropna(axis=0, how='any')
trips_data_frame = trips_data_frame.reset_index(drop=True)

In [3]:
data = open('Datasets/stations.xml').read()
xml_root = ET.XML(data) 
all_stations = []
for i, child_node in enumerate(xml_root):
    stations = {}
    for child in child_node:
        stations[child.tag] = child.text
    all_stations.append(stations)
stations = pd.DataFrame(all_stations)
stations2 = pd.DataFrame(all_stations)
stations['coords'] = list(zip(stations.lat, stations.long))
stations = stations[['id','name','coords']]

In [4]:
count_stations = trips_data_frame.groupby(['start_station_name','end_station_name']).size().reset_index(name='rides')
stations_rides_network = pd.merge(count_stations, stations, how='inner', left_on=['start_station_name'], right_on=['name'])
stations_rides_network = pd.merge(stations_rides_network, stations, how='inner', left_on=['end_station_name'], right_on=['name'])
stations_rides_network = stations_rides_network[['start_station_name','end_station_name','rides','coords_x','coords_y']]
stations_rides_network['distribution'] = stations_rides_network.apply(lambda row: geopy.distance.vincenty(row['coords_x'], row['coords_y']).miles, axis=1)

stations_count = trips_data_frame.groupby(['start_station_name','end_station_name']).size().reset_index(name='rides')
stations_rides_network1 = pd.merge(stations_count, stations2, how='inner', left_on=['start_station_name'], right_on=['name'])
stations_rides_network1 = pd.merge(stations_rides_network, stations2, how='inner', left_on=['end_station_name'], right_on=['name'])


In [7]:
stations2[['lat','long']] = stations2[['lat','long']].apply(pd.to_numeric)
map = folium.Map([51.51, -0.078], zoom_start=12)
for index, row in stations2.iterrows():
    folium.CircleMarker([row['lat'], row['long']],
                        radius=3,
                        popup=row['terminalName'],
                        fill_color="#e43d53",
                       ).add_to(map)

In [9]:
station_list = stations2[['lat', 'long']].as_matrix()
map.add_children(plugins.HeatMap(station_list, radius=30))
map